In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.colors
import statsmodels.formula.api as smf
import seaborn as sns
from cem.match import match
from cem.coarsen import coarsen
from cem.imbalance import L1
import statsmodels.api as sm


## Removing countries

In [2]:
matplotlib.rc('font', family='Arial') 

In [57]:

pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df=pop_df[['Country Code','2019']].dropna()
pop_df['2019']=pop_df['2019'].astype(int)
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values
possible_countries=[x.lower() for x in possible_countries]

excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS",   # Timor-Leste
    "GEO", #Georgia
    'SWZ'
]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]


possible_iso=list(set(possible_countries)-set(excluded_iso3_codes))

## Load data

[2002, 2003, 2004, 2005, 2006, 2007, 2014, 2015, 2016, 2017, 2018, 2019]

In [ ]:
df = pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df=df[df['country'].isin(possible_iso)]
df.rename(columns={'aggregated_value': 'count', 'country': 'Mention_country', 'affiliation_country': 'Aff_country'}, inplace=True)
df = df[df['year'].isin(range(2002, 2020))]
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=['MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
df=df[~df['subjarea'].isin(physical_sciences)]
df['post']=df['year'].apply(lambda x: 0 if x>=2002 and x<=2010 else 1 )

df_sample=df.groupby(['Mention_country','post'])['count'].sum().reset_index()
df_sample['count']/=9
df_pivoted = df_sample.pivot(index=['Mention_country'], columns='post', values='count').reset_index().fillna(0)
df_pivoted.rename(columns={0:'count_pre_ArabSpring', 1:'count_post_ArabSpring'}, inplace=True)
df_pivoted['difference']=(df_pivoted['count_post_ArabSpring']-df_pivoted['count_pre_ArabSpring'])
df_pivoted['rank_pre_ArabSpring']=df_pivoted['count_pre_ArabSpring'].rank(pct=True)
df_pivoted['rank_post_ArabSpring']=df_pivoted['count_post_ArabSpring'].rank(pct=True)
df_pivoted['rank_difference']=df_pivoted['difference'].rank(pct=True)

In [72]:
def find_matches(c):
    rank_country=df_pivoted[df_pivoted['Mention_country']==c]['rank_pre_ArabSpring'].values[0]
    return df_pivoted[(df_pivoted['rank_pre_ArabSpring']>=rank_country-0.05) &(df_pivoted['rank_pre_ArabSpring']<=rank_country+0.05)]['Mention_country'].values

In [76]:
c='syr'
rank_country=df_pivoted[df_pivoted['Mention_country']==c]['rank_pre_ArabSpring'].values[0]
df_pivoted[(df_pivoted['rank_pre_ArabSpring']>=rank_country-0.05) &(df_pivoted['rank_pre_ArabSpring']<=rank_country+0.05)].sort_values('difference', ascending=False)

post,Mention_country,count_pre_ArabSpring,count_post_ArabSpring,difference,rank_pre_ArabSpring,rank_post_ArabSpring,rank_difference
66,kaz,112.896859,632.723186,519.826327,0.349315,0.561644,0.650685
86,mmr,77.196137,319.336071,242.139935,0.294521,0.431507,0.506849
0,afg,115.536695,346.630504,231.093809,0.356164,0.458904,0.493151
69,khm,105.351546,318.336474,212.984929,0.335616,0.424658,0.479452
115,rwa,60.768206,247.524972,186.756767,0.267123,0.349315,0.458904
125,syr,83.770240,264.513715,180.743475,0.315068,0.369863,0.438356
111,pse,75.001943,207.520965,132.519022,0.287671,0.315068,0.369863
79,lva,67.431872,192.813098,125.381226,0.280822,0.294521,0.363014
16,bih,94.528593,211.505914,116.977321,0.321918,0.321918,0.349315
11,ben,81.426255,188.525629,107.099374,0.301370,0.280822,0.315068


In [ ]:

for c in abbr:
    matched=find_matches(c)
    possible_match=list(set(matched)-set(abbr))
    possible_match=possible_match+[c]
    print(possible_match)
    df_analysis=df_sample[df_sample['Mention_country'].isin(possible_match)]
    df_analysis['treated']=df_analysis['Mention_country']==c
    df_analysis['treated']=df_analysis['treated'].astype(int)
    model = smf.ols('count ~ treated + post + treated:post', data=df_analysis).fit()
    print(model.summary())


['dnk', 'mys', 'vnm', 'che', 'ken', 'bel', 'pak', 'col', 'cze', 'aut', 'sgp', 'prt', 'hun', 'idn', 'egy']
                            OLS Regression Results                            
Dep. Variable:                  count   R-squared:                       0.524
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     9.539
Date:                Wed, 05 Mar 2025   Prob (F-statistic):           0.000201
Time:                        15:24:06   Log-Likelihood:                -226.58
No. Observations:                  30   AIC:                             461.2
Df Residuals:                      26   BIC:                             466.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29156\4101673591.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['treated']=df_analysis['Mention_country']==c
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29156\4101673591.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis['treated']=df_analysis['treated'].astype(int)
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_29156\4101673591.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us